# TPCH - 1

In [1]:
from datetime import datetime, timedelta
import pysdql

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)

d1 = datetime.strptime('1998-12-01', "%Y-%m-%d")
d3 = d1 + timedelta(days=76)
var1 = d3.strftime("%Y-%m-%d")

r = lineitem[(lineitem['l_shipdate'] <= var1)]
r = r.groupby(['l_returnflag', 'l_linestatus'])
r = r.aggr(sum_qty=(r['l_quantity'], 'sum'),
           sum_base_price=(r['l_extendedprice'], 'sum'),
           sum_disc_price=(r['l_extendedprice'] * (1 - r['l_discount']), 'sum'),
           sum_charge=(r['l_extendedprice'] * (1 - r['l_discount']) * (1 + r['l_tax']), 'sum'),
           avg_qty=(r['l_quantity'], 'avg'),
           avg_price=(r['l_extendedprice'], 'avg'),
           avg_disc=(r['l_discount'], 'avg'),
           count_order=('*', 'count'))

db_driver.run(r)

let lineitem = load[{<l_orderkey: int, l_partkey: int, l_suppkey: int, l_linenumber: int, l_quantity: int, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_shipinstruct: string, l_shipmode: string, l_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let lmp = sum (<lin_k, lin_v> in lineitem) if ((lin_k.l_shipdate <= date(19990215))) then { lin_k -> 1 } else {  }
let tmpa = sum (<l_k, l_v> in lmp) { < l_returnflag=l_k.l_returnflag, l_linestatus=l_k.l_linestatus > -> { l_k -> l_v } }
let tmp = sum (<ta_k, ta_v> in tmpa) { < l_returnflag=ta_k.l_returnflag, l_linestatus=ta_k.l_linestatus, group=ta_v > -> 1 }
let tmpb = sum (<t_k, t_v> in tmp) sum (<g_k, g_v> in t_k.group) { < l_returnflag=t_k.l_returnflag, l_linestatus=t_k.l_linestatus > -> < sum_qty=g_k.l_quantity * g_v, sum_base_price=g_k.l_extendedprice * g_v, sum_disc_price=(g_k.l_extendedprice * (1 - g_k.l_d

{<l_returnflag = N, l_linestatus = O, sum_qty = 77372, sum_base_price = 7.759263143000013E7, sum_disc_price = 7.375810409310006E7, sum_charge = 7.670202845039211E7, avg_qty = 25.518469656992085, avg_price = 25591.237279023786, avg_disc = 0.04971635883905017, count_order = 3032> -> 1, <l_returnflag = A, l_linestatus = F, sum_qty = 37474, sum_base_price = 3.7569624639999956E7, sum_disc_price = 3.567619209700001E7, sum_charge = 3.710141622242408E7, avg_qty = 25.35453315290934, avg_price = 25419.231826792937, avg_disc = 0.05086603518267946, count_order = 1478> -> 1, <l_returnflag = R, l_linestatus = F, sum_qty = 36511, sum_base_price = 3.657084124000005E7, sum_disc_price = 3.47384728758E7, sum_charge = 3.6169060112192966E7, avg_qty = 25.059025394646532, avg_price = 25100.096938915613, avg_disc = 0.050027453671928776, count_order = 1457> -> 1, <l_returnflag = N, l_linestatus = F, sum_qty = 1041, sum_base_price = 1041301.0700000003, sum_disc_price = 999060.8979999999, sum_charge = 1036450.80

# TPCH - 3

In [2]:
import pysdql

var1 = 'BUILDING'
var2 = '1995-03-22'

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

customer = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/customer.tbl', header=pysdql.CUSTOMER_COLS)
orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)
lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)

r = customer.merge(orders, on=(customer['c_custkey'] == orders['o_custkey']))
r = r.merge(lineitem, on=(r['o_orderkey'] == lineitem['l_orderkey']))

r = r[(r['c_mktsegment'] == var1)
      & (r['o_orderdate'] < var2)
      & (r['l_shipdate'] > var2)]

r = r.groupby(['l_orderkey', 'o_orderdate', 'o_shippriority']) \
    .aggr(revenue=((r['l_extendedprice'] * (1 - r['l_discount'])), 'sum'))

db_driver.run(r)

let lineitem = load[{<l_orderkey: int, l_partkey: int, l_suppkey: int, l_linenumber: int, l_quantity: int, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_shipinstruct: string, l_shipmode: string, l_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let orders = load[{<o_orderkey: int, o_custkey: int, o_orderstatus: string, o_totalprice: double, o_orderdate: date, o_orderpriority: string, o_clerk: string, o_shippriority: int, o_comment: string> -> int}]("T:/UG4-Proj/datasets/orders.tbl")
let customer = load[{<c_custkey: int, c_name: string, c_address: string, c_nationkey: int, c_phone: string, c_acctbal: double, c_mktsegment: string, c_comment: string> -> int}]("T:/UG4-Proj/datasets/customer.tbl")
let Rm0 = sum (<cus_k, cus_v> in customer) sum (<ord_k, ord_v> in orders) if ((cus_k.c_custkey == ord_k.o_custkey)) then { concat(cus_k, ord_k) -> cus_v * ord_v } e

{<l_orderkey = 3492, o_orderdate = DateValue(19941124), o_shippriority = 0, revenue = 43716.072400000005> -> 1, <l_orderkey = 1637, o_orderdate = DateValue(19950208), o_shippriority = 0, revenue = 87438.8481> -> 1, <l_orderkey = 775, o_orderdate = DateValue(19950318), o_shippriority = 0, revenue = 55713.6063> -> 1, <l_orderkey = 998, o_orderdate = DateValue(19941126), o_shippriority = 0, revenue = 6811.434> -> 1, <l_orderkey = 742, o_orderdate = DateValue(19941223), o_shippriority = 0, revenue = 43728.048> -> 1, <l_orderkey = 5191, o_orderdate = DateValue(19941211), o_shippriority = 0, revenue = 49378.309400000006> -> 1, <l_orderkey = 3430, o_orderdate = DateValue(19941212), o_shippriority = 0, revenue = 4726.6775> -> 1, <l_orderkey = 2883, o_orderdate = DateValue(19950123), o_shippriority = 0, revenue = 36666.9612> -> 1}

# TPCH - 4

In [3]:
import pysdql

var1 = '1996-05-01'
var2 = '1996-08-01'  # var1 + 3 month

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)

r = lineitem[(lineitem['l_commitdate'] < lineitem['l_receiptdate'])]
r = r.merge(orders, on=(r['l_orderkey'] == orders['o_orderkey']))

s = orders[(orders['o_orderdate'] >= var1) & (orders['o_orderdate'] < var2) & r.exists()]

s = s.groupby(['o_orderpriority']).aggr(order_count=('*', 'count'))

db_driver.run(s)

let orders = load[{<o_orderkey: int, o_custkey: int, o_orderstatus: string, o_totalprice: double, o_orderdate: date, o_orderpriority: string, o_clerk: string, o_shippriority: int, o_comment: string> -> int}]("T:/UG4-Proj/datasets/orders.tbl")
let lineitem = load[{<l_orderkey: int, l_partkey: int, l_suppkey: int, l_linenumber: int, l_quantity: int, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_shipinstruct: string, l_shipmode: string, l_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let lmp = sum (<lin_k, lin_v> in lineitem) if ((lin_k.l_commitdate < lin_k.l_receiptdate)) then { lin_k -> 1 } else {  }
let Rm0 = sum (<l_k, l_v> in lmp) sum (<ord_k, ord_v> in orders) if ((l_k.l_orderkey == ord_k.o_orderkey)) then { concat(l_k, ord_k) -> l_v * ord_v } else {  }
let count_Rm0 = sum (<r0_k, r0_v> in Rm0) r0_v
let omp = sum (<ord_k, ord_v> in orders) if ((((da

{<o_orderpriority = 1-URGENT, order_count = 13> -> 1, <o_orderpriority = 4-NOT SPECIFIED, order_count = 12> -> 1, <o_orderpriority = 5-LOW, order_count = 7> -> 1, <o_orderpriority = 2-HIGH, order_count = 7> -> 1, <o_orderpriority = 3-MEDIUM, order_count = 9> -> 1}

# TPCH - 5

In [4]:
import pysdql

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

var1 = 'MIDDLE EAST'
var2 = '1993-01-01'
var3 = '1994-01-01'  # var2 + 1 year

customer = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/customer.tbl', header=pysdql.CUSTOMER_COLS)
orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)
lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
supplier = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/supplier.tbl', header=pysdql.SUPPLIER_COLS)
nation = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/nation.tbl', header=pysdql.NATION_COLS)
region = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/region.tbl', header=pysdql.REGION_COLS)

r = customer.merge(right=orders, on=(customer['c_custkey'] == orders['o_custkey']))
r = r.merge(lineitem, on=(r['o_orderkey'] == lineitem['l_orderkey']))
r = r.merge(supplier, on=((r['l_suppkey'] == supplier['s_suppkey'])
                          & (r['c_nationkey'] == supplier['s_nationkey'])
                          )
            )
r = r.merge(nation, on=(r['s_nationkey'] == nation['n_nationkey']))
r = r.merge(region, on=(r['n_regionkey'] == region['r_regionkey']))

r = r[(region['r_name'] == var1)
      & (orders['o_orderdate'] >= var2)
      & (orders['o_orderdate'] < var3)]

r = r.groupby(['n_name']).aggr(revenue=((lineitem['l_extendedprice'] * (1 - lineitem['l_discount'])), 'sum'))

db_driver.run(r)

let region = load[{<r_regionkey: int, r_name: string, r_comment: string> -> int}]("T:/UG4-Proj/datasets/region.tbl")
let nation = load[{<n_nationkey: int, n_name: string, n_regionkey: int, n_comment: string> -> int}]("T:/UG4-Proj/datasets/nation.tbl")
let supplier = load[{<s_suppkey: int, s_name: string, s_address: string, s_nationkey: int, s_phone: string, s_acctbal: double, s_comment: string> -> int}]("T:/UG4-Proj/datasets/supplier.tbl")
let lineitem = load[{<l_orderkey: int, l_partkey: int, l_suppkey: int, l_linenumber: int, l_quantity: int, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_shipinstruct: string, l_shipmode: string, l_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let orders = load[{<o_orderkey: int, o_custkey: int, o_orderstatus: string, o_totalprice: double, o_orderdate: date, o_orderpriority: string, o_clerk: string, o_shippriority: in

{<n_name = IRAN, revenue = 183980.59919999997> -> 1, <n_name = IRAQ, revenue = 194852.39520000003> -> 1}

# TPCH - 6

In [5]:
import pysdql

var1 = '1993-01-01'
var2 = '1994-01-01'  # var1 + 1 year
discount = 0.07
var3 = discount - 0.01
var4 = discount + 0.01
var5 = 25

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)

r = lineitem[(lineitem['l_shipdate'] >= var1) & (lineitem['l_shipdate'] < var2)
             & (lineitem['l_discount'] > var3) & (lineitem['l_discount'] < var4)
             & (lineitem['l_quantity'] < var5)]

r = r.aggr(revenue=(r['l_extendedprice'] * r['l_discount'], 'sum'))

db_driver.run(r)

let lineitem = load[{<l_orderkey: int, l_partkey: int, l_suppkey: int, l_linenumber: int, l_quantity: int, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_shipinstruct: string, l_shipmode: string, l_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let lmp = sum (<lin_k, lin_v> in lineitem) if ((((((date(19930101) <= lin_k.l_shipdate) && (lin_k.l_shipdate < date(19940101))) && (0.060000000000000005 < lin_k.l_discount)) && (lin_k.l_discount < 0.08)) && (lin_k.l_quantity < 25))) then { lin_k -> 1 } else {  }
let tmp = sum (<l_k, l_v> in lmp) < revenue=(l_k.l_extendedprice * l_k.l_discount) * l_v >
let tmpa = { < revenue=tmp.revenue > -> 1 }
tmpa
[info] welcome to sbt 1.6.0 (Oracle Corporation Java 17)
[info] loading project definition from T:\sdql\project
[info] loading settings for project sdql from build.sbt ...
[info] set current project to sdql (in build f

{<revenue = 53485.054300000025> -> 1}

# TPCH - 7

In [6]:
import pysdql

var1 = 'MOZAMBIQUE'
var2 = 'JORDAN'

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

supplier = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/supplier.tbl', header=pysdql.SUPPLIER_COLS)
lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)
customer = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/customer.tbl', header=pysdql.CUSTOMER_COLS)
n1_cols = ['n1_nationkey', 'n1_name', 'n1_regionkey', 'n1_comment']
n2_cols = ['n2_nationkey', 'n2_name', 'n2_regionkey', 'n2_comment']
n1 = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/nation.tbl', header=n1_cols, name='n1')
n2 = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/nation.tbl', header=n2_cols, name='n2')
r1 = supplier.merge(n1, on=(supplier['s_nationkey'] == n1['n1_nationkey'])).rename('r1')

r2 = customer.merge(n2, on=(customer['c_nationkey'] == n2['n2_nationkey'])).rename('r2')

r = r1.merge(r2, on=((r1['n1_name'] == 'MOZAMBIQUE') & (r2['n2_name'] == 'JORDAN'))
                        | ((r1['n1_name'] == 'JORDAN') & (r2['n2_name'] == 'MOZAMBIQUE')))

r = r.merge(lineitem, on=(r['s_suppkey'] == lineitem['l_suppkey']))
r = r.merge(orders, on=(r['l_orderkey'] == orders['o_orderkey']) & (r['c_custkey'] == orders['o_custkey']))

r = r[(r['l_shipdate'] >= '1995-01-01') & (r['l_shipdate'] <= '1996-12-31')]

r['supp_nation'] = r['n1_name']
r['cust_nation'] = r['n2_name']
r['volume'] = r['l_extendedprice'] * (1 - r['l_discount'])
r['l_year'] = r['l_shipdate'].year

r = r[['supp_nation', 'cust_nation', 'l_year', 'volume']].rename('shiping')

r = r.groupby(['supp_nation', 'cust_nation', 'l_year']).aggr(revenue=(r['volume'], 'sum'))

db_driver.run(r)

let orders = load[{<o_orderkey: int, o_custkey: int, o_orderstatus: string, o_totalprice: double, o_orderdate: date, o_orderpriority: string, o_clerk: string, o_shippriority: int, o_comment: string> -> int}]("T:/UG4-Proj/datasets/orders.tbl")
let lineitem = load[{<l_orderkey: int, l_partkey: int, l_suppkey: int, l_linenumber: int, l_quantity: int, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_shipinstruct: string, l_shipmode: string, l_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let n2 = load[{<n2_nationkey: int, n2_name: string, n2_regionkey: int, n2_comment: string> -> int}]("T:/UG4-Proj/datasets/nation.tbl")
let customer = load[{<c_custkey: int, c_name: string, c_address: string, c_nationkey: int, c_phone: string, c_acctbal: double, c_mktsegment: string, c_comment: string> -> int}]("T:/UG4-Proj/datasets/customer.tbl")
let Rm0 = sum (<cus_k, cus_v>

[error] java.lang.Exception: Error: `sum(<Sym(r2_k),Sym(r2_v)> <- ZeroValue) ...` doesn't have a dictionary range: `class sdql.ir.ZeroValue$`[error] 	at sdql.package$.raise(package.scala:6)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:118)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:21)[error] 	at sdql.backend.Interpreter$.apply(Interpreter.scala:11)[error] 	at sdql.driver.Main$.main(Main.scala:19)[error] 	at sdql.driver.Main.main(Main.scala)[error] 	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)[error] 	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)[error] 	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)[error] 	at java.base/java.lang.reflect.Method.invoke(Method.java:568)[error] stack trace is suppressed; run 'last Compile / run' for the full output[error] (Compile / run) java.lang.Exception: Error: `sum(<Sym(

# TPCH - 8

In [7]:
var1 = 'JORDAN'
var2 = 'MIDDLE EAST'
var3 = 'SMALL ANODIZED COPPER'
db_driver = pysdql.db_driver(db_path=r'T:/sdql')

part = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/part.tbl', header=pysdql.PART_COLS)
supplier = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/supplier.tbl', header=pysdql.SUPPLIER_COLS)
lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)
customer = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/customer.tbl', header=pysdql.CUSTOMER_COLS)
region = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/region.tbl', header=pysdql.REGION_COLS)
n1_cols = ['n1_nationkey', 'n1_name', 'n1_regionkey', 'n1_comment']
n2_cols = ['n2_nationkey', 'n2_name', 'n2_regionkey', 'n2_comment']
n1 = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/nation.tbl', header=n1_cols, name='n1')
n2 = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/nation.tbl', header=n2_cols, name='n2')

r1 = customer.merge(n1, on=customer['c_nationkey'] == n1['n1_nationkey'])
r1 = r1.merge(region, on=r1['n1_regionkey'] == region['r_regionkey'])

r1 = r1[(r1['r_name'] == var2)]

r1 = r1.merge(orders, on=r1['c_custkey'] == orders['o_custkey'])

r1 = r1[(r1['o_orderdate'] >= '1995-01-01') & (r1['o_orderdate'] <= '1996-12-31')]

r1 = r1.merge(lineitem, on=r1['o_orderkey'] == lineitem['l_orderkey']).rename('r1')
r2 = supplier.merge(n2, on=supplier['s_nationkey'] == n2['n2_nationkey']).rename('r2')

r1 = r1.merge(r2, on=r1['l_suppkey'] == r2['s_suppkey'])
r = r1.merge(part, on=r1['l_partkey'] == part['p_partkey'])

r = r[(r['p_type'] == var3)]

r['o_year'] = r['o_orderdate'].year
r['volume'] = r['l_extendedprice'] * (1 - r['l_discount'])
r['nation'] = r['n2_name']

r = r[['o_year', 'volume', 'nation']]
all_nations = r.rename('all_nations')

all_nations['mkt_value'] = all_nations.case(all_nations['nation'] == var1, all_nations['volume'], 0)
all_nations['mkt_value'] = all_nations['mkt_value'] / all_nations['volume']

s = all_nations.groupby(['o_year']).aggr(mkt_share=(all_nations['mkt_value'], 'sum'))

db_driver.run(s)

let part = load[{<p_partkey: int, p_name: string, p_mfgr: string, p_brand: string, p_type: string, p_size: int, p_container: string, p_retailprice: double, p_comment: string> -> int}]("T:/UG4-Proj/datasets/part.tbl")
let n2 = load[{<n2_nationkey: int, n2_name: string, n2_regionkey: int, n2_comment: string> -> int}]("T:/UG4-Proj/datasets/nation.tbl")
let supplier = load[{<s_suppkey: int, s_name: string, s_address: string, s_nationkey: int, s_phone: string, s_acctbal: double, s_comment: string> -> int}]("T:/UG4-Proj/datasets/supplier.tbl")
let Rm0 = sum (<sup_k, sup_v> in supplier) sum (<n2_k, n2_v> in n2) if ((sup_k.s_nationkey == n2_k.n2_nationkey)) then { concat(sup_k, n2_k) -> sup_v * n2_v } else {  }
let r2 = Rm0
let lineitem = load[{<l_orderkey: int, l_partkey: int, l_suppkey: int, l_linenumber: int, l_quantity: int, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_sh

{<o_year = 1995, mkt_share = 0> -> 1}

# TPCH - 9

In [22]:
var1 = 'cornflower'

part = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/part.tbl', header=pysdql.PART_COLS)
supplier = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/supplier.tbl', header=pysdql.SUPPLIER_COLS)
lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
partsupp = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/partsupp.tbl', header=pysdql.PARTSUPP_COLS)
orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)
nation = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/nation.tbl', header=pysdql.NATION_COLS)

# part_p
sub_p = part[part['p_name'].startswith(var1)].rename('sub_p')

# hash join (part, partsupp)
r1 = sub_p.merge(partsupp, on=sub_p['p_partkey'] == partsupp['ps_partkey']).rename('r1')

# hash join (supplier, nation)
r2 = supplier.merge(nation, on=(supplier['s_nationkey'] == nation['n_nationkey']))

# hash join ((supplier, nation), (part, partsupp))
r2 = r2.merge(r1, on=(r2['s_suppkey'] == r1['ps_suppkey'])).rename('r2')
r = r2.merge(lineitem, on=(r2['s_suppkey'] == lineitem['l_suppkey']) & (r2['ps_suppkey'] == lineitem['l_suppkey']))
r = r.merge(orders, on=(r['l_orderkey'] == orders['o_orderkey'])).rename('r')

r['nation'] = r['n_name']
r['o_year'] = r['o_orderdate'].year
r['amount'] = r['l_extendedprice'] * (1 - r['l_discount']) - r['ps_supplycost'] * r['l_quantity']

profit = r[['nation', 'o_year', 'amount']].rename('profit')

s = profit.groupby(['nation', 'o_year']).aggr(sum_profit=(profit['amount'], 'sum'))

pysdql.db_driver(db_path=r'T:/sdql').run(s)

let orders = load[{<o_orderkey: int, o_custkey: int, o_orderstatus: string, o_totalprice: double, o_orderdate: date, o_orderpriority: string, o_clerk: string, o_shippriority: int, o_comment: string> -> int}]("T:/UG4-Proj/datasets/orders.tbl")
let lineitem = load[{<l_orderkey: int, l_partkey: int, l_suppkey: int, l_linenumber: int, l_quantity: int, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_shipinstruct: string, l_shipmode: string, l_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let partsupp = load[{<ps_partkey: int, ps_suppkey: int, ps_availqty: int, ps_supplycost: double, ps_comment: string> -> int}]("T:/UG4-Proj/datasets/partsupp.tbl")
let part = load[{<p_partkey: int, p_name: string, p_mfgr: string, p_brand: string, p_type: string, p_size: int, p_container: string, p_retailprice: double, p_comment: string> -> int}]("T:/UG4-Proj/datasets/part.tbl"

{<nation = UNITED KINGDOM, o_year = 1993, sum_profit = 1255630.9392000001> -> 1, <nation = IRAN, o_year = 1996, sum_profit = 4247711.794399998> -> 1, <nation = IRAQ, o_year = 1996, sum_profit = 2867742.3636000003> -> 1, <nation = PERU, o_year = 1993, sum_profit = 1483256.8842> -> 1, <nation = ARGENTINA, o_year = 1995, sum_profit = 1642866.8516> -> 1, <nation = IRAN, o_year = 1994, sum_profit = 3328476.553399998> -> 1, <nation = IRAQ, o_year = 1995, sum_profit = 2111850.346199999> -> 1, <nation = IRAQ, o_year = 1992, sum_profit = 2501160.7879999992> -> 1, <nation = UNITED KINGDOM, o_year = 1996, sum_profit = 1506323.1003999994> -> 1, <nation = IRAQ, o_year = 1993, sum_profit = 2250890.355600001> -> 1, <nation = IRAN, o_year = 1995, sum_profit = 3201627.9369999995> -> 1, <nation = ARGENTINA, o_year = 1994, sum_profit = 1682303.1093999993> -> 1, <nation = IRAQ, o_year = 1998, sum_profit = 1527687.5272> -> 1, <nation = ETHIOPIA, o_year = 1993, sum_profit = 131003.24009999982> -> 1, <nation

# TPCH - 10

In [9]:
var1 = '1993-08-01'
var2 = '1993-11-01'  # var1 + 3 month

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

customer = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/customer.tbl', header=pysdql.CUSTOMER_COLS)
orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)
lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
nation = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/nation.tbl', header=pysdql.NATION_COLS)

part_o = orders[(orders['o_orderdate'] >= var1) & (orders['o_orderdate'] < var2)].rename('part_o')

r = customer.merge(part_o, on=customer['c_custkey'] == part_o['o_custkey'])

r = r.merge(nation, on=r['c_nationkey'] == nation['n_nationkey'])

part_l = lineitem[lineitem['l_returnflag'] == 'R'].rename('part_l')

r = r.merge(part_l, on=r['o_custkey'] == part_l['l_orderkey'])

r = r.groupby(['c_custkey', 'c_name', 'c_acctbal', 'c_phone', 'n_name', 'c_address', 'c_comment'])\
    .aggr(revenue=((r['l_extendedprice'] * (1 - r['l_discount'])), 'sum'))
db_driver.run(r)

let lineitem = load[{<l_orderkey: int, l_partkey: int, l_suppkey: int, l_linenumber: int, l_quantity: int, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_shipinstruct: string, l_shipmode: string, l_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let lmp = sum (<lin_k, lin_v> in lineitem) if ((lin_k.l_returnflag == "R")) then { lin_k -> 1 } else {  }
let part_l = lmp
let nation = load[{<n_nationkey: int, n_name: string, n_regionkey: int, n_comment: string> -> int}]("T:/UG4-Proj/datasets/nation.tbl")
let orders = load[{<o_orderkey: int, o_custkey: int, o_orderstatus: string, o_totalprice: double, o_orderdate: date, o_orderpriority: string, o_clerk: string, o_shippriority: int, o_comment: string> -> int}]("T:/UG4-Proj/datasets/orders.tbl")
let omp = sum (<ord_k, ord_v> in orders) if (((date(19930801) <= ord_k.o_orderdate) && (ord_k.o_orderdate < date(1993110

{<c_custkey = 97, c_name = Customer#000000097, c_acctbal = 2164.48, c_phone = 27-588-919-5638, n_name = PERU, c_address = OApyejbhJG,0Iw3j rd1M, c_comment = haggle slyly. bold, special ideas are blithely above the thinly bold theo, revenue = 61459.7804> -> 1, <c_custkey = 70, c_name = Customer#000000070, c_acctbal = 4867.52, c_phone = 32-828-107-2832, n_name = RUSSIA, c_address = mFowIuhnHjp2GjCiYYavkW kUwOjIaTCQ, c_comment = fter the special asymptotes. ideas after the unusual frets cajole quickly regular pinto be, revenue = 40120.1103> -> 1, <c_custkey = 131, c_name = Customer#000000131, c_acctbal = 8595.53, c_phone = 21-840-210-3572, n_name = IRAQ, c_address = jyN6lAjb1FtH10rMC,XzlWyCBrg75, c_comment = jole special packages. furiously final dependencies about the furiously speci, revenue = 43260.479999999996> -> 1, <c_custkey = 64, c_name = Customer#000000064, c_acctbal = -646.64, c_phone = 13-558-731-7204, n_name = CANADA, c_address = MbCeGY20kaKK3oalJD,OT, c_comment = structions a

# TPCH - 11

In [10]:
var1 = 'PERU'
var2 = 0.0001

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

partsupp = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/partsupp.tbl', header=pysdql.PARTSUPP_COLS)
supplier = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/supplier.tbl', header=pysdql.SUPPLIER_COLS)
nation = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/nation.tbl', header=pysdql.NATION_COLS)

sub_n = nation[(nation['n_name'] == var1)].rename('sub_n')

join_ns = sub_n.merge(supplier, on=(sub_n['n_nationkey'] == supplier['s_nationkey'])).rename('join_ns')
r1 = join_ns.merge(partsupp, join_ns['s_suppkey'] == partsupp['ps_suppkey']).rename('r1')

agg_val = (r1['ps_supplycost'] * r1['ps_availqty'] * 1).sum()

# GOURPBY HAVING
r = r1.groupby(['ps_partkey']).filter(lambda x: (x['ps_supplycost'] * x['ps_availqty']).sum() > agg_val)

# SELECT GROUPBY AGGREGATION
r = r.groupby(['ps_partkey']).aggr(value=(r['ps_supplycost'] * r['ps_availqty'], 'sum'))

db_driver.run(r)

let partsupp = load[{<ps_partkey: int, ps_suppkey: int, ps_availqty: int, ps_supplycost: double, ps_comment: string> -> int}]("T:/UG4-Proj/datasets/partsupp.tbl")
let supplier = load[{<s_suppkey: int, s_name: string, s_address: string, s_nationkey: int, s_phone: string, s_acctbal: double, s_comment: string> -> int}]("T:/UG4-Proj/datasets/supplier.tbl")
let nation = load[{<n_nationkey: int, n_name: string, n_regionkey: int, n_comment: string> -> int}]("T:/UG4-Proj/datasets/nation.tbl")
let nmp = sum (<nat_k, nat_v> in nation) if ((nat_k.n_name == "PERU")) then { nat_k -> 1 } else {  }
let sub_n = nmp
let Rm0 = sum (<sn_k, sn_v> in sub_n) sum (<sup_k, sup_v> in supplier) if ((sn_k.n_nationkey == sup_k.s_nationkey)) then { concat(sn_k, sup_k) -> sn_v * sup_v } else {  }
let join_ns = Rm0
let Rm1 = sum (<jn_k, jn_v> in join_ns) sum (<psup_k, psup_v> in partsupp) if ((jn_k.s_suppkey == psup_k.ps_suppkey)) then { concat(jn_k, psup_k) -> jn_v * psup_v } else {  }
let r1 = Rm1
let agg_val = su

{<ps_partkey = 97, value = 4314022.47> -> 1, <ps_partkey = 164, value = 2643604.15> -> 1, <ps_partkey = 171, value = 1998922.56> -> 1, <ps_partkey = 72, value = 2023966.94> -> 1, <ps_partkey = 62, value = 2056642.72> -> 1, <ps_partkey = 44, value = 607304.7000000001> -> 1, <ps_partkey = 141, value = 231038.80000000002> -> 1, <ps_partkey = 17, value = 1.3534598E7> -> 1, <ps_partkey = 170, value = 4371681.399999999> -> 1, <ps_partkey = 50, value = 1036069.2799999999> -> 1, <ps_partkey = 122, value = 3226213.8800000004> -> 1, <ps_partkey = 135, value = 873132.22> -> 1, <ps_partkey = 166, value = 4239796.54> -> 1, <ps_partkey = 40, value = 7662503.61> -> 1, <ps_partkey = 150, value = 4245428.61> -> 1, <ps_partkey = 148, value = 1424237.3699999999> -> 1, <ps_partkey = 194, value = 626466.0> -> 1, <ps_partkey = 179, value = 751076.4999999999> -> 1, <ps_partkey = 94, value = 2687324.8800000004> -> 1, <ps_partkey = 42, value = 107882.52> -> 1, <ps_partkey = 56, value = 5440869.45> -> 1, <ps_pa

# TPCH - 12

In [11]:
var1 = 'FOB'
var2 = 'AIR'
var3 = '1995-01-01'
var4 = '1996-01-01'

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)
lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)

sub_l = lineitem[(lineitem['l_shipmode'].isin((var1, var2)))
                 & (lineitem['l_commitdate'] < lineitem['l_receiptdate'])
                 & (lineitem['l_shipdate'] < lineitem['l_commitdate'])
                 & (lineitem['l_receiptdate'] >= var3) & (lineitem['l_receiptdate'] < var4)].rename('sub_l')

r = orders.merge(sub_l, on=(orders['o_orderkey'] == sub_l['l_orderkey']))

r['high_line_priority'] = r.case((r['o_orderpriority'] == '1-URGENT') | (r['o_orderpriority'] == '2-HIGH'), 1, 0)
r['low_line_priority'] = r.case((r['o_orderpriority'] != '1-URGENT') | (r['o_orderpriority'] != '2-HIGH'), 1, 0)

r = r.groupby(['l_shipmode']).aggr(high_line_count=(r['high_line_priority'], 'sum'),
                                   low_line_count=(r['low_line_priority'], 'sum'))

db_driver.run(r)

let lineitem = load[{<l_orderkey: int, l_partkey: int, l_suppkey: int, l_linenumber: int, l_quantity: int, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_shipinstruct: string, l_shipmode: string, l_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let lmp = sum (<lin_k, lin_v> in lineitem) if (((((((lin_k.l_shipmode == "AIR") || (lin_k.l_shipmode == "FOB")) && (lin_k.l_commitdate < lin_k.l_receiptdate)) && (lin_k.l_shipdate < lin_k.l_commitdate)) && (date(19950101) <= lin_k.l_receiptdate)) && (lin_k.l_receiptdate < date(19960101)))) then { lin_k -> 1 } else {  }
let sub_l = lmp
let orders = load[{<o_orderkey: int, o_custkey: int, o_orderstatus: string, o_totalprice: double, o_orderdate: date, o_orderpriority: string, o_clerk: string, o_shippriority: int, o_comment: string> -> int}]("T:/UG4-Proj/datasets/orders.tbl")
let Rm0 = sum (<ord_k, ord_v> in orders) 

{<l_shipmode = FOB, high_line_count = 6, low_line_count = 12> -> 1, <l_shipmode = AIR, high_line_count = 8, low_line_count = 13> -> 1}

# TPCH - 13

In [21]:
var1 = 'special'
var2 = 'requests'

orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)
customer = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/customer.tbl', header=pysdql.CUSTOMER_COLS)

sub_o = orders[orders['o_comment'].str.not_contains(var1, var2)].rename('sub_o')

# LEFT OUTER JOIN ?
r = customer.merge(sub_o, on=(customer['c_custkey'] == sub_o['o_custkey']))

c_orders = r.groupby(['c_custkey']).aggr(c_count=(r['o_orderkey'], 'count')).rename('c_orders')

s = c_orders.groupby(['c_count']).aggr(custdist=('*', 'count'))

pysdql.db_driver(db_path=r'T:/sdql').run(s)

let orders = load[{<o_orderkey: int, o_custkey: int, o_orderstatus: string, o_totalprice: double, o_orderdate: date, o_orderpriority: string, o_clerk: string, o_shippriority: int, o_comment: string> -> int}]("T:/UG4-Proj/datasets/orders.tbl")
let omp = sum (<ord_k, ord_v> in orders) if (!(ext(`StrContainsN`, ord_k.o_comment, "special", "requests"))) then { ord_k -> 1 } else {  }
let sub_o = omp
let customer = load[{<c_custkey: int, c_name: string, c_address: string, c_nationkey: int, c_phone: string, c_acctbal: double, c_mktsegment: string, c_comment: string> -> int}]("T:/UG4-Proj/datasets/customer.tbl")
let Rm0 = sum (<cus_k, cus_v> in customer) sum (<so_k, so_v> in sub_o) if ((cus_k.c_custkey == so_k.o_custkey)) then { concat(cus_k, so_k) -> cus_v * so_v } else {  }
let tmp = sum (<r0_k, r0_v> in Rm0) { < c_custkey=r0_k.c_custkey > -> { r0_k -> r0_v } }
let rmp = sum (<t_k, t_v> in tmp) { < c_custkey=t_k.c_custkey, group=t_v > -> 1 }
let tmpa = sum (<r_k, r_v> in rmp) sum (<g_k, g_v>

{<c_count = 26, custdist = 3> -> 1, <c_count = 29, custdist = 2> -> 1, <c_count = 5, custdist = 3> -> 1, <c_count = 7, custdist = 4> -> 1, <c_count = 8, custdist = 4> -> 1, <c_count = 22, custdist = 4> -> 1, <c_count = 9, custdist = 6> -> 1, <c_count = 16, custdist = 9> -> 1, <c_count = 15, custdist = 2> -> 1, <c_count = 4, custdist = 3> -> 1, <c_count = 6, custdist = 3> -> 1, <c_count = 18, custdist = 4> -> 1, <c_count = 17, custdist = 6> -> 1, <c_count = 21, custdist = 3> -> 1, <c_count = 19, custdist = 3> -> 1, <c_count = 12, custdist = 8> -> 1, <c_count = 11, custdist = 3> -> 1, <c_count = 28, custdist = 1> -> 1, <c_count = 24, custdist = 2> -> 1, <c_count = 13, custdist = 4> -> 1, <c_count = 23, custdist = 4> -> 1, <c_count = 10, custdist = 6> -> 1, <c_count = 3, custdist = 1> -> 1, <c_count = 14, custdist = 5> -> 1, <c_count = 25, custdist = 1> -> 1, <c_count = 20, custdist = 6> -> 1}

# TPCH - 14

In [13]:
var1 = '1995-03-01'
var2 = '1995-04-01'  # var1 + 1 month

lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
part = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/part.tbl', header=pysdql.PART_COLS)

sub_l = lineitem[(lineitem['l_shipdate'] >= var1) & (lineitem['l_shipdate'] < var2)].rename('sub_l')
r = sub_l.merge(part, on=sub_l['l_partkey'] == part['p_partkey'])

r['promo'] = r.case(r['p_type'].startswith('PROMO'), r['l_extendedprice'] * (1 - r['l_discount']), 0)

r = r.aggr(promo_revenue=(100 * r['promo'] / (r['l_extendedprice'] * (1 - r['l_discount'])), 'sum'))

pysdql.db_driver(db_path=r'T:/sdql').run(r)

let part = load[{<p_partkey: int, p_name: string, p_mfgr: string, p_brand: string, p_type: string, p_size: int, p_container: string, p_retailprice: double, p_comment: string> -> int}]("T:/UG4-Proj/datasets/part.tbl")
let lineitem = load[{<l_orderkey: int, l_partkey: int, l_suppkey: int, l_linenumber: int, l_quantity: int, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_shipinstruct: string, l_shipmode: string, l_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let lmp = sum (<lin_k, lin_v> in lineitem) if (((date(19950301) <= lin_k.l_shipdate) && (lin_k.l_shipdate < date(19950401)))) then { lin_k -> 1 } else {  }
let sub_l = lmp
let Rm0 = sum (<sl_k, sl_v> in sub_l) sum (<par_k, par_v> in part) if ((sl_k.l_partkey == par_k.p_partkey)) then { concat(sl_k, par_k) -> sl_v * par_v } else {  }
let rmp = sum (<r0_k, r0_v> in Rm0) if (ext(`StrStartsWith`, r0_k.p_t

{<promo_revenue = 700.0> -> 1}

# TPCH - 16

In [14]:
var1 = 'Brand#21'
var2 = 'SMALL ANODIZED'
var3 = (48, 33, 18, 16, 8, 3, 10, 42)

supplier = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/supplier.tbl', header=pysdql.SUPPLIER_COLS)
part = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/part.tbl', header=pysdql.PART_COLS)
partsupp = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/partsupp.tbl', header=pysdql.PARTSUPP_COLS)

sub_s = supplier[supplier['s_comment'].contains('Customer', 'Complaints')].rename('sub_s')

sub_p = part[(part['p_brand'] != 'Brand#21')
             & (~part['p_type'].str.startswith('SMALL ANODIZED'))
             & (part['p_size'].isin(var3))].rename('sub_p')

r = sub_p.merge(partsupp, on=sub_p['p_partkey'] == partsupp['ps_partkey'])

r = r[~(r['ps_suppkey'].isin(sub_s['s_suppkey']))]

# COUNT (DISTINCT value) ?
r = r.groupby(['p_brand', 'p_type', 'p_size']).aggr(supplier_cnt=(r['ps_suppkey'], 'sum'))

pysdql.db_driver(db_path=r'T:/sdql').run(r, block=False)

let supplier = load[{<s_suppkey: int, s_name: string, s_address: string, s_nationkey: int, s_phone: string, s_acctbal: double, s_comment: string> -> int}]("T:/UG4-Proj/datasets/supplier.tbl")
let smp = sum (<sup_k, sup_v> in supplier) if (ext(`StrContainsN`, sup_k.s_comment, "Customer", "Complaints")) then { sup_k -> 1 } else {  }
let sub_s = smp
let partsupp = load[{<ps_partkey: int, ps_suppkey: int, ps_availqty: int, ps_supplycost: double, ps_comment: string> -> int}]("T:/UG4-Proj/datasets/partsupp.tbl")
let part = load[{<p_partkey: int, p_name: string, p_mfgr: string, p_brand: string, p_type: string, p_size: int, p_container: string, p_retailprice: double, p_comment: string> -> int}]("T:/UG4-Proj/datasets/part.tbl")
let pmp = sum (<par_k, par_v> in part) if ((((par_k.p_brand != "Brand#21") && !(ext(`StrStartsWith`, par_k.p_type, "SMALL ANODIZED"))) && ((((((((par_k.p_size == 42) || (par_k.p_size == 10)) || (par_k.p_size == 48)) || (par_k.p_size == 33)) || (par_k.p_size == 18)) || (p

[error] java.lang.Exception: Error: `sum(<Sym(t_k),Sym(t_v)> <- ZeroValue) ...` doesn't have a dictionary range: `class sdql.ir.ZeroValue$`[error] 	at sdql.package$.raise(package.scala:6)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:118)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:21)[error] 	at sdql.backend.Interpreter$.apply(Interpreter.scala:11)[error] 	at sdql.driver.Main$.main(Main.scala:19)[error] 	at sdql.driver.Main.main(Main.scala)[error] 	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)[error] 	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)[error] 	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)[error] 	at java.base/java.lang.reflect.Method.invoke(Method.java:568)[error] stack trace is suppressed; run 'last Compile / run' for the full output[error] (Compile / run) java.lang.Exception: Error: `sum(<Sym(t_

# TPCH - 17

In [15]:
var1 = 'Brand#11'
var2 = 'WRAP CAN'

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
part = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/part.tbl', header=pysdql.PART_COLS)

part_agg = lineitem.groupby(['l_partkey']) \
    .aggr(agg_partkey=lineitem['l_partkey'], avg_quantity=(0.2 * lineitem['l_quantity'], 'avg')) \
    .rename('part_agg')

sub_p = part[(part['p_brand'] == var1) & (part['p_container'] == var2)].rename('sub_p')

r = sub_p.merge(part_agg, on=sub_p['p_partkey'] == part_agg['agg_partkey'])

r = r[(r['l_quantity'] < r['avg_quantity'])]

r = r.aggr(avg_yearly=((lineitem['l_extendedprice'] / 7.0), 'sum'))

db_driver.run(r)

let lineitem = load[{<l_orderkey: int, l_partkey: int, l_suppkey: int, l_linenumber: int, l_quantity: int, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_shipinstruct: string, l_shipmode: string, l_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let tmp = sum (<lin_k, lin_v> in lineitem) { < l_partkey=lin_k.l_partkey > -> { lin_k -> lin_v } }
let lmp = sum (<t_k, t_v> in tmp) { < l_partkey=t_k.l_partkey, group=t_v > -> 1 }
let tmpa = sum (<l_k, l_v> in lmp) sum (<g_k, g_v> in l_k.group) { < l_partkey=l_k.l_partkey > -> < avg_quantity_sum=(0.2 * g_k.l_quantity) * g_v, avg_quantity_count=g_v > }
let aggr_lineitem = sum (<ta_k, ta_v> in tmpa) { < agg_partkey=ta_k.l_partkey, avg_quantity=(ta_v.avg_quantity_sum / ta_v.avg_quantity_count) > -> 1 }
let part_agg = aggr_lineitem
let part = load[{<p_partkey: int, p_name: string, p_mfgr: string, p_brand: string, p_t

[error] java.lang.Exception: Error: `sum(<Sym(r0_k),Sym(r0_v)> <- ZeroValue) ...` doesn't have a dictionary range: `class sdql.ir.ZeroValue$`[error] 	at sdql.package$.raise(package.scala:6)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:118)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:21)[error] 	at sdql.backend.Interpreter$.apply(Interpreter.scala:11)[error] 	at sdql.driver.Main$.main(Main.scala:19)[error] 	at sdql.driver.Main.main(Main.scala)[error] 	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)[error] 	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)[error] 	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)[error] 	at java.base/java.lang.reflect.Method.invoke(Method.java:568)[error] stack trace is suppressed; run 'last Compile / run' for the full output[error] (Compile / run) java.lang.Exception: Error: `sum(<Sym(

# TPCH - 18

In [24]:
var1 = 231

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

customer = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/customer.tbl', header=pysdql.CUSTOMER_COLS)
orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)
lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)

aggr_l = lineitem.groupby(['l_orderkey']).filter(lambda x: x['l_quantity'].sum() > var1)

sub_o = orders[(orders['o_orderkey'].isin(aggr_l['l_orderkey']))]

r = customer.merge(sub_o, on=(customer['c_custkey'] == sub_o['o_custkey']))
r = r.merge(lineitem, on=r['o_orderkey'] == lineitem['l_orderkey'])

r = r.groupby(['c_name', 'c_custkey', 'o_orderkey', 'o_orderdate', 'o_totalprice'])\
    .aggr({r['l_quantity']: 'sum'})

db_driver.run(r)

let lineitem = load[{<l_orderkey: int, l_partkey: int, l_suppkey: int, l_linenumber: int, l_quantity: int, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_shipinstruct: string, l_shipmode: string, l_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let tmp = sum (<lin_k, lin_v> in lineitem) { < l_orderkey=lin_k.l_orderkey > -> { lin_k -> lin_v } }
let lmp = sum (<t_k, t_v> in tmp) { < l_orderkey=t_k.l_orderkey, group=t_v > -> 1 }
let hvmp = sum (<l_k, l_v> in lmp) sum (<g_k, g_v> in l_k.group) { < l_orderkey=l_k.l_orderkey > -> < val=(g_k.l_quantity * g_v) > }
let hvR = sum (<hv_k, hv_v> in hvmp) if ((231 < hv_v.val)) then { < l_orderkey=hv_k.l_orderkey > -> 1 } else {  }
let fhv_lineitem = sum (<lin_k, lin_v> in lineitem) sum (<h_k, h_v> in hvR) if ((h_k.l_orderkey == lin_k.l_orderkey)) then { lin_k -> 1 } else {  }
let orders = load[{<o_orderkey: int, o_cu

{<c_name = Customer#000000076, c_custkey = 76, o_orderkey = 5158, o_orderdate = DateValue(19970121), o_totalprice = 240284.95, l_quantity_sum = 248> -> 1, <c_name = Customer#000000076, c_custkey = 76, o_orderkey = 1477, o_orderdate = DateValue(19970824), o_totalprice = 231831.35, l_quantity_sum = 236> -> 1, <c_name = Customer#000000131, c_custkey = 131, o_orderkey = 4484, o_orderdate = DateValue(19961224), o_totalprice = 237947.61, l_quantity_sum = 243> -> 1, <c_name = Customer#000000146, c_custkey = 146, o_orderkey = 5925, o_orderdate = DateValue(19951113), o_totalprice = 242588.87, l_quantity_sum = 242> -> 1, <c_name = Customer#000000068, c_custkey = 68, o_orderkey = 2208, o_orderdate = DateValue(19950501), o_totalprice = 245388.06, l_quantity_sum = 256> -> 1, <c_name = Customer#000000029, c_custkey = 29, o_orderkey = 1121, o_orderdate = DateValue(19970113), o_totalprice = 241837.88, l_quantity_sum = 242> -> 1, <c_name = Customer#000000010, c_custkey = 10, o_orderkey = 4421, o_orderd

# TPCH - 19

In [17]:
var1 = 'Brand#54'
var2 = 'Brand#22'
var3 = 'Brand#12'
var4 = 1
var4_1 = var4 + 10
var5 = 18
var5_1 = var5 + 10
var6 = 26
var6_1 = var6 + 10

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
part = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/part.tbl', header=pysdql.PART_COLS)

a1 = (part['p_brand'] == var1) \
     & (part['p_container'].isin(('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG'))) \
     & (part['p_size'] >= 1) & (part['p_size'] <= 5)

a2 = (part['p_brand'] == var2) \
     & (part['p_container'].isin(('MED BAG', 'MED BOX', 'MED PKG', 'MED PACK'))) \
     & (part['p_size'] >= 1) & (part['p_size'] <= 10)

a3 = (part['p_brand'] == 'Brand#13') \
     & (part['p_container'].isin(('LG CASE', 'LG BOX', 'LG PACK', 'LG PKG'))) \
     & (part['p_size'] >= 1) & (part['p_size'] <= 15)

sub_p = part[a1 | a2 | a3].rename('sub_p')

b1 = (lineitem['l_quantity'] >= var4) & (lineitem['l_quantity'] <= var4_1) \
     & (lineitem['l_shipmode'].isin(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')

b2 = (lineitem['l_quantity'] >= var5) & (lineitem['l_quantity'] <= var5_1) \
     & (lineitem['l_shipmode'].isin(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')

b3 = (lineitem['l_quantity'] >= var6) \
     & (lineitem['l_quantity'] <= var6_1) \
     & (lineitem['l_shipmode'].isin(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')

sub_l = lineitem[b1 | b2 | b3].rename('sub_l')

r = sub_l.merge(sub_p, on=(sub_l['l_partkey'] == sub_p['p_partkey']))

r = r.aggr(revenue=((r['l_extendedprice'] * (1 - r['l_discount'])), 'sum'))

db_driver.run(r)

let part = load[{<p_partkey: int, p_name: string, p_mfgr: string, p_brand: string, p_type: string, p_size: int, p_container: string, p_retailprice: double, p_comment: string> -> int}]("T:/UG4-Proj/datasets/part.tbl")
let pmp = sum (<par_k, par_v> in part) if (((((((par_k.p_brand == "Brand#54") && ((((par_k.p_container == "SM PKG") || (par_k.p_container == "SM PACK")) || (par_k.p_container == "SM CASE")) || (par_k.p_container == "SM BOX"))) && (1 <= par_k.p_size)) && (par_k.p_size <= 5)) || ((((par_k.p_brand == "Brand#22") && ((((par_k.p_container == "MED PACK") || (par_k.p_container == "MED PKG")) || (par_k.p_container == "MED BAG")) || (par_k.p_container == "MED BOX"))) && (1 <= par_k.p_size)) && (par_k.p_size <= 10))) || ((((par_k.p_brand == "Brand#13") && ((((par_k.p_container == "LG PKG") || (par_k.p_container == "LG PACK")) || (par_k.p_container == "LG CASE")) || (par_k.p_container == "LG BOX"))) && (1 <= par_k.p_size)) && (par_k.p_size <= 15)))) then { par_k -> 1 } else {  }
let 

{<revenue = 865.92> -> 1}

# TPCH - 20

In [18]:
var1 = 'orange'
var2 = '1994-01-01'
var2_1 = '1995-01-01'  # var2 + 1 year
var3 = 'FRANCE'

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
part = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/part.tbl', header=pysdql.PART_COLS)
partsupp = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/partsupp.tbl', header=pysdql.PARTSUPP_COLS)
supplier = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/supplier.tbl', header=pysdql.SUPPLIER_COLS)
nation = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/nation.tbl', header=pysdql.NATION_COLS)

sub_l = lineitem[(lineitem['l_shipdate'] >= var2) & (lineitem['l_shipdate'] < var2_1)].rename('sub_l')
agg_lineitem = sub_l.groupby(['l_partkey', 'l_suppkey']) \
    .aggr(agg_partkey=sub_l['l_partkey'],
          agg_suppkey=sub_l['l_suppkey'],
          agg_quantity=(0.5 * sub_l['l_quantity'], 'sum')) \
    .rename('agg_lineitem')

sub_p = part[part['p_name'].startswith(var1)].rename('sub_p')

r = partsupp.merge(agg_lineitem, on=(partsupp['ps_partkey'] == agg_lineitem['agg_partkey'])
                                    & (partsupp['ps_suppkey'] == agg_lineitem['agg_suppkey']))

r = r[(r['ps_partkey'].isin(sub_p['p_partkey']) & (r['ps_availqty'] > r['agg_quantity']))]

sub_n = nation[(nation['n_name'] == var3)].rename('sub_n')

s = supplier.merge(sub_n, on=supplier['s_nationkey'] == sub_n['n_nationkey'])

s = s[(s['s_suppkey'].isin(r['ps_suppkey']))]

s = s[['s_name', 's_address']]

db_driver.run(supplier)

let supplier = load[{<s_suppkey: int, s_name: string, s_address: string, s_nationkey: int, s_phone: string, s_acctbal: double, s_comment: string> -> int}]("T:/UG4-Proj/datasets/supplier.tbl")
let Rm0 = sum (<sup_k, sup_v> in supplier) sum (<sn_k, sn_v> in sub_n) if ((sup_k.s_nationkey == sn_k.n_nationkey)) then { concat(sup_k, sn_k) -> sup_v * sn_v } else {  }
supplier
[info] welcome to sbt 1.6.0 (Oracle Corporation Java 17)
[info] loading project definition from T:\sdql\project
[info] loading settings for project sdql from build.sbt ...
[info] set current project to sdql (in build file:/T:/sdql/)
[info]
[info] Here are some highlights of this release:
[info]   - Improved JDK 17 support
[info]   - Improved Build Server Protocol (BSP) support
[info]   - Tab completion of global keys
[info] See https://eed3si9n.com/sbt-1.6.0 for full release notes.
[info] Hide the banner for this release by running `skipBanner`.
[error] server failed to start on local:sbt-server-ec9dcaf7bfc5935dead3. jav

[error] java.lang.Exception: Error: Variable `sub_n` not in scope![error] 	at sdql.package$.raise(package.scala:6)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:17)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:105)[error] 	at sdql.backend.Interpreter$.$anonfun$run$8(Interpreter.scala:113)[error] 	at sdql.backend.Interpreter$.$anonfun$run$8$adapted(Interpreter.scala:112)[error] 	at scala.collection.immutable.HashMap$HashMap1.foreach(HashMap.scala:399)[error] 	at scala.collection.immutable.HashMap$HashTrieMap.foreach(HashMap.scala:725)[error] 	at scala.collection.immutable.HashMap$HashTrieMap.foreach(HashMap.scala:725)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:112)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:21)[error] 	at sdql.backend.Interpreter$.apply(Interpreter.scala:11)[error] 	at sdql.driver.Main$.main(Main.scala:19)[error] 	at sdql.driver.Main.main(Main.scala)[error] 	at java.base/jdk.internal.reflect.NativeMethodAccessor

# TPCH - 21

In [19]:
var1 = 'INDIA'

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

supplier = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/supplier.tbl', header=pysdql.SUPPLIER_COLS)
l1_cols = ['l1_orderkey', 'l1_partkey', 'l1_suppkey', 'l1_linenumber', 'l1_quantity', 'l1_extendedprice',
           'l1_discount', 'l1_tax', 'l1_returnflag', 'l1_linestatus', 'l1_shipdate', 'l1_commitdate',
           'l1_receiptdate', 'l1_shipinstruct', 'l1_shipmode', 'l1_comment']
l1 = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=l1_cols, name='l1')
l2_cols = ['l2_orderkey', 'l2_partkey', 'l2_suppkey', 'l2_linenumber', 'l2_quantity', 'l2_extendedprice',
           'l2_discount', 'l2_tax', 'l2_returnflag', 'l2_linestatus', 'l2_shipdate', 'l2_commitdate',
           'l2_receiptdate', 'l2_shipinstruct', 'l2_shipmode', 'l2_comment']
l2 = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=l2_cols, name='l2')
l3_cols = ['l3_orderkey', 'l3_partkey', 'l3_suppkey', 'l3_linenumber', 'l3_quantity', 'l3_extendedprice',
           'l3_discount', 'l3_tax', 'l3_returnflag', 'l3_linestatus', 'l3_shipdate', 'l3_commitdate',
           'l3_receiptdate', 'l3_shipinstruct', 'l3_shipmode', 'l3_comment']
l3 = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=l3_cols, name='l3')
orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)
nation = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/nation.tbl', header=pysdql.NATION_COLS)

sub_n = nation[(nation['n_name'] == var1)].rename('sub_n')

join_ns = supplier.merge(sub_n, on=supplier['s_nationkey'] == sub_n['n_nationkey']).rename('join_ns')

sub_l1 = l1[(l1['l1_receiptdate'] > l1['l1_commitdate'])]

r = join_ns.merge(sub_l1, on=(join_ns['s_suppkey'] == sub_l1['l1_suppkey']))

sub_o = orders[(orders['o_orderstatus'] == 'F')]

r = r.merge(sub_o, on=(r['l1_orderkey'] == sub_o['o_orderkey']))

r1 = l2.merge(sub_l1, on=((l2['l2_orderkey'] == sub_l1['l1_orderkey'])
                          & (l2['l2_suppkey'] != sub_l1['l1_suppkey']))
              ).rename('r1')

sub_l3 = l3[(l3['l3_receiptdate'] > l3['l3_commitdate'])]

r2 = l3.merge(sub_l1,
              on=((l3['l3_orderkey'] == sub_l1['l1_orderkey'])
                  & (l3['l3_suppkey'] != sub_l1['l1_suppkey']))
              ).rename('r2')

s = r[r1.exists() & r2.not_exists()]

s = s.groupby(['s_name']).aggr(numwait=('*', 'count'))

db_driver.run(r)

let l1 = load[{<l1_orderkey: int, l1_partkey: int, l1_suppkey: int, l1_linenumber: int, l1_quantity: int, l1_extendedprice: double, l1_discount: double, l1_tax: double, l1_returnflag: string, l1_linestatus: string, l1_shipdate: date, l1_commitdate: date, l1_receiptdate: date, l1_shipinstruct: string, l1_shipmode: string, l1_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let lmp = sum (<l1_k, l1_v> in l1) if ((l1_k.l1_commitdate < l1_k.l1_receiptdate)) then { l1_k -> 1 } else {  }
let l3 = load[{<l3_orderkey: int, l3_partkey: int, l3_suppkey: int, l3_linenumber: int, l3_quantity: int, l3_extendedprice: double, l3_discount: double, l3_tax: double, l3_returnflag: string, l3_linestatus: string, l3_shipdate: date, l3_commitdate: date, l3_receiptdate: date, l3_shipinstruct: string, l3_shipmode: string, l3_comment: string> -> int}]("T:/UG4-Proj/datasets/lineitem.tbl")
let lmp = sum (<l3_k, l3_v> in l3) if ((l3_k.l3_commitdate < l3_k.l3_receiptdate)) then { l3_k -> 1 } else {  

[error] java.lang.Exception: Error: `RecordValue(ArrayBuffer((l3_orderkey,3264), (l3_partkey,131), (l3_suppkey,2), (l3_linenumber,2), (l3_quantity,34), (l3_extendedprice,35058.42), (l3_discount,0.0), (l3_tax,0.010000000000000002), (l3_returnflag,N), (l3_linestatus,O), (l3_shipdate,DateValue(19970103)), (l3_commitdate,DateValue(19970106)), (l3_receiptdate,DateValue(19970129)), (l3_shipinstruct,TAKE BACK RETURN), (l3_shipmode,REG AIR), (l3_comment,rns haggle carefully. blit))).l1_orderkey`: not field named `l1_orderkey`[error] 	at sdql.package$.raise(package.scala:6)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:72)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:45)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:38)[error] 	at sdql.backend.Interpreter$.run(Interpreter.scala:38)[error] 	at sdql.backend.Interpreter$.$anonfun$run$8(Interpreter.scala:113)[error] 	at sdql.backend.Interpreter$.$anonfun$run$8$adapted(Interpreter.scala:112)[error] 	at scala.c

# TPCH - 22

In [20]:
var1 = ('16', '12', '18', '14', '30', '27', '25')

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

customer = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/customer.tbl', header=pysdql.CUSTOMER_COLS)

sub_c = customer[(customer['c_acctbal'] > 0.00)
                 & (customer['c_phone'].str.substring(0, 2).isin(var1))]

db_driver.run(sub_c)

let customer = load[{<c_custkey: int, c_name: string, c_address: string, c_nationkey: int, c_phone: string, c_acctbal: double, c_mktsegment: string, c_comment: string> -> int}]("T:/UG4-Proj/datasets/customer.tbl")
let cmp = sum (<cus_k, cus_v> in customer) if (((0.0 < cus_k.c_acctbal) && (((((((ext(`SubString`, cus_k.c_phone, 0, 2) == "25") || (ext(`SubString`, cus_k.c_phone, 0, 2) == "27")) || (ext(`SubString`, cus_k.c_phone, 0, 2) == "16")) || (ext(`SubString`, cus_k.c_phone, 0, 2) == "12")) || (ext(`SubString`, cus_k.c_phone, 0, 2) == "18")) || (ext(`SubString`, cus_k.c_phone, 0, 2) == "14")) || (ext(`SubString`, cus_k.c_phone, 0, 2) == "30")))) then { cus_k -> 1 } else {  }
cmp
[info] welcome to sbt 1.6.0 (Oracle Corporation Java 17)
[info] loading project definition from T:\sdql\project
[info] loading settings for project sdql from build.sbt ...
[info] set current project to sdql (in build file:/T:/sdql/)
[info]
[info] Here are some highlights of this release:
[info]   - Improved 

{<c_custkey = 115, c_name = Customer#000000115, c_address = 0WFt1IXENmUT2BgbsB0ShVKJZt0HCBCbFl0aHc, c_nationkey = 8, c_phone = 18-971-699-1843, c_acctbal = 7508.92, c_mktsegment = HOUSEHOLD, c_comment = sits haggle above the carefully ironic theodolite> -> 1, <c_custkey = 53, c_name = Customer#000000053, c_address = HnaxHzTfFTZs8MuCpJyTbZ47Cm4wFOOgib, c_nationkey = 15, c_phone = 25-168-852-5363, c_acctbal = 4113.64, c_mktsegment = HOUSEHOLD, c_comment = ar accounts are. even foxes are blithely. fluffily pending deposits boost> -> 1, <c_custkey = 50, c_name = Customer#000000050, c_address = 9SzDYlkzxByyJ1QeTI o, c_nationkey = 6, c_phone = 16-658-112-3221, c_acctbal = 4266.13, c_mktsegment = MACHINERY, c_comment = ts. furiously ironic accounts cajole furiously slyly ironic dinos.> -> 1, <c_custkey = 54, c_name = Customer#000000054, c_address = ,k4vf 5vECGWFy,hosTE,, c_nationkey = 4, c_phone = 14-776-370-4745, c_acctbal = 868.9, c_mktsegment = AUTOMOBILE, c_comment = sual, silent accounts